## Sentiment Classification Using Proposed CNN-GNN on MR Polarity dataset + Token Sentiment Injection + Dataset Partitioning

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from nltk.tokenize import TweetTokenizer
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import time
from torch_scatter import scatter_max, scatter_mean, scatter_std
from sklearn.model_selection import KFold
import torchmetrics
import lightning as L
from torch_geometric.data import Batch, Data
from tqdm import tqdm
from nltk.stem import PorterStemmer
import spacy
import string
import os
import re
import math
import pickle 
from torch import nn
from torch.utils.flop_counter import FlopCounterMode
from transformers import AutoModel, DebertaV2Tokenizer, AutoTokenizer
import matplotlib.pyplot as plt
from torch_geometric.nn import GATv2Conv, summary
from tqdm import tqdm
from sklearn.metrics import classification_report, f1_score, accuracy_score, precision_score, recall_score, confusion_matrix, hinge_loss
from torchmetrics import ConfusionMatrix
from codes.scripts.utilities import ClassifierModelManager, CnnGnnClassifierLightningModel
from os import path
from pathlib import Path

In [114]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-large", cache_dir=r'cache_dir/tokenizers/')
id_vocab = {v:k for k,v in tokenizer.vocab.items()}

f:\Articles\CGNet\venv\Lib\site-packages\transformers\convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [115]:
import re
stem = PorterStemmer()

def clean(text):
    text = text.lower()
    text = re.sub('[^a-zA-Z0-9]',' ',text)
    text = text.split()
    text = [ i for i in text if i not in stopwords.words('english')]
    text = [stem.stem(i) for i in text]
    return ' '.join(text)

In [116]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 100
num_embedding=12288
device

device(type='cuda')

In [117]:
# def load_dataset(dataset_path, shuffle = False):
#     pos_path = os.path.join(dataset_path, 'pos')
#     neg_path = os.path.join(dataset_path, 'neg')
#     pos_names = os.listdir(pos_path)
#     neg_names = os.listdir(neg_path)
#     df = []
#     for pname in pos_names:
#         file_path = os.path.join(pos_path, pname)
#         with open(file_path, 'rt', encoding='utf8') as f:
#             df.append([1, f.read()])
    
#     for nname in neg_names:
#         file_path = os.path.join(neg_path, nname)
#         with open(file_path, 'rt', encoding='utf8') as f:
#             df.append([0, f.read()])
#     df = pd.DataFrame(data=df, columns=['label', 'text'])
#     if shuffle:
#         df = df.sample(frac=1).reset_index(drop=True)
#     return df

# keep_ratio = 1.0
# df = load_dataset(r'Data\TextClassification\review_polarity')
# # df.text = df.text.apply(lambda d: d[:50])
# df.dropna(inplace=True)
# df = df.iloc[:int(keep_ratio*df.shape[0])]
# df['clean'] = df['text'].apply(clean)
# df.to_csv(r"Data\TextClassification\review_polarity\df-clean.csv")

df = pd.read_csv(r"Data\TextClassification\review_polarity\df-clean.csv")
df.drop(['Unnamed: 0', 'text'], axis=1, inplace=True)
target_classes = ["Negative", "Positive"]
df.shape

(2000, 2)

In [118]:
class_list = df.label.unique()
class_id = {target_classes[i]:i for i in class_list}
id_class = {i:target_classes[i] for i in class_list}

In [137]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold

class KFoldDataset(Dataset):
    def __init__(self, X, y, num_classes, tokenizer,  batch_size=128, shuffle=True, k_folds=10, fold_idx=0):
        super().__init__()
        self.fold_idx = fold_idx
        kf = KFold(k_folds, shuffle=shuffle, random_state=42)
        
        # self.kfolds1 = {}
        # for fold, (train_ids, test_ids) in enumerate(kf.split(X, y)):
        #     self.kfolds1[fold] = (train_ids, test_ids)
        self.k_folds = list(kf.split(np.arange(len(y))))
        self.k_folds = [(np.array(f[0], dtype=np.longlong), np.array(f[1], dtype=np.longlong)) for f in self.k_folds]
        if batch_size > len(y)//k_folds:
            batch_size = len(y)//k_folds
        self.batch_size = batch_size
        
        if len(y) % batch_size*k_folds != 0:
            
            self.shortage = ((len(y) // (batch_size*k_folds))+1)*(batch_size*k_folds) - len(y)
            print(f"!!!! This amount of data added: {self.shortage}, change batch size or k-fold to reduce it!")
            empty_labels = [i%2 for i in range(self.shortage)]
            empty_strings = [id_class[l] for l in empty_labels]
            y = np.concatenate([y, empty_labels])
            X = np.concatenate([X, empty_strings])
        
        self.y = torch.from_numpy(y)
        self.shuffle = shuffle
        # self.y = torch.nn.functional.one_hot(self.y, num_classes=num_classes).float()
        self.X = X
        self.tokenizer = tokenizer
        sent_len = max([len(x.split()) for x in X])
        print(f'sent_len: {sent_len}')
        self.all_data =  torch.tensor([tokenizer(doc, padding='max_length', truncation=True, max_length=min(sent_len, 1024)).input_ids for doc in X])
            
    def update_split(self, fold_idx):
        self.active_fold = self.k_folds[fold_idx]
        
        self.train_y = self.y[self.active_fold[0]]
        self.test_y = self.y[self.active_fold[1]]
        
        self.all_train_data = self.all_data[self.active_fold[0]]
        self.all_test_data = self.all_data[self.active_fold[1]]
        self.train()
        
    def train(self):
        self.is_train=True
        
    def eval(self):
        self.is_train=False
        
    def __getitem__(self, index):
        if self.is_train:
            return self.all_train_data[index], self.train_y[index]
        else:
            return self.all_test_data[index], self.test_y[index]
    
    def __len__(self):
        return len(self.train_y) if self.is_train else len(self.test_y)

In [138]:
class TrainTestDatasetWrapper(Dataset):
    
    def __init__(self, target_dataset, is_train=True) -> None:
        super().__init__()
        self.dataset = target_dataset
        self.is_train = is_train
    
    def __getitem__(self, index):
        if self.is_train:
            if not self.dataset.is_train:
                self.dataset.train()
            return self.dataset[index]
        else:
            if self.dataset.is_train:
                self.dataset.eval()
            return self.dataset[index]
    
    def set_active_fold(self, fold_idx):
        self.dataset.update_split(fold_idx)
    
    def reset_params(self):
        self.dataset.reset_params()
        
    def __len__(self):
        if self.is_train:
            return len(self.dataset.train_y)
        else:
            return len(self.dataset.test_y)

In [139]:
main_dataset = KFoldDataset(df.clean.values, df.label.values, len(class_id), tokenizer, batch_size=batch_size, shuffle=True, k_folds=10)
batch_size = main_dataset.batch_size

sent_len: 1394


In [140]:
train_dataset = TrainTestDatasetWrapper(main_dataset, True)
test_dataset = TrainTestDatasetWrapper(main_dataset, False)
train_dataset.set_active_fold(0)
test_dataset.set_active_fold(0)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, drop_last=False, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, drop_last=False, shuffle=True)

In [141]:
X, y = next(iter(test_dataloader))

## Model Classes and Functions

In [142]:
# Model
class GRUTextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes):
        super(GRUTextClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.bigru1 = nn.GRU(embed_dim, 128, batch_first=True, bidirectional=True)
        self.bigru2 = nn.GRU(128*2, 64, batch_first=True, bidirectional=True)
        self.global_max_pool = nn.AdaptiveMaxPool1d(1)
        self.fc1 = nn.Linear(64*2, 256)
        self.dropout1 = nn.Dropout(0.25)
        self.fc2 = nn.Linear(256, 128)
        self.dropout2 = nn.Dropout(0.25)
        self.fc3 = nn.Linear(128, 64)
        self.dropout3 = nn.Dropout(0.25)
        self.out = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.embedding(x)                         # [B, L] -> [B, L, E]
        x, _ = self.bigru1(x)                          # [B, L, 2*128]
        x, _ = self.bigru2(x)                          # [B, L, 2*64]
        x = x.permute(0, 2, 1)                         # [B, 2*64, L]
        x = self.global_max_pool(x).squeeze(-1)        # [B, 2*64]
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = F.relu(self.fc3(x))
        x = self.dropout3(x)
        return self.out(x)


In [154]:
# Example values (adjust as needed)
vocabulary_size = tokenizer.vocab_size
embed_size = 256
num_classes = 2
num_epochs = 70
lr = 0.0012
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [155]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from torchmetrics import ConfusionMatrix

def calculate_metrics(cl_model, dataloader, num_classes, save_path):
    cm = ConfusionMatrix(task="multiclass", num_classes=num_classes)

    y_pred = []
    y_true = []

    cl_model = cl_model.eval()
    cl_model.to(device)
    for X, y in tqdm(dataloader):
        X = X.to(device)
        with torch.no_grad():
            y_p = cl_model(X)
            y_p = y_p.cpu()
        y_pred.append(y_p)
        y_true.append(y)
    y_pred = torch.cat(y_pred, dim=0)
    y_true = torch.cat(y_true, dim=0)
    y_pred2 = torch.argmax(y_pred, dim=1)
    y_true2 = torch.argmax(y_true, dim=1) if len(y_true.shape)> 1 else y_true
    
    file_path = Path(save_path)
    file_path.parent.mkdir(parents=True, exist_ok=True)
    with open(file_path, 'at+') as f:
        print(f'classification report: \n {classification_report(y_true2, y_pred2, digits=4)}', file=f)
        print(f'confusion matrix:\n {cm(y_pred2, y_true2)}', file=f)
        print('================================', file=f)


In [156]:
# Loss and optimizer

def train_model(num_epochs, train_dataloader, test_dataloader, fold_i=0):
    model = GRUTextClassifier(vocabulary_size, embed_size, num_classes)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0.00001)
    # Training Loop
    model = model.to(device)
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for inputs, labels in tqdm(train_dataloader):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)                     # [B, num_classes]
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # Validation
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for val_inputs, val_labels in tqdm(test_dataloader):
                val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
                val_outputs = model(val_inputs)
                v_loss = loss_fn(val_outputs, val_labels)
                val_loss += v_loss.item()
                preds = val_outputs.argmax(dim=1)
                correct += (preds == val_labels).sum().item()
                total += val_labels.size(0)
        val_acc = correct / total if total > 0 else 0

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        
    calculate_metrics(model, test_dataloader, num_classes, save_path=rf"logs\rt2k-bi-gru\fold-{fold_i}.txt")

In [157]:
for i in range(10):
    train_dataset.set_active_fold(i)
    test_dataset.set_active_fold(i)
    # max_token_count = max(train_dataset.max_token_count, test_dataset.max_token_count)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, drop_last=False, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, drop_last=False, shuffle=True)
    train_model(num_epochs, train_dataloader, test_dataloader, fold_i=i)
    time.sleep(20)
    torch.cuda.empty_cache()
    import gc
    gc.collect()
    time.sleep(20)

100%|██████████| 2/2 [00:00<00:00, 52.64it/s]


Epoch 1/70, Loss: 12.5382, Val Loss: 1.3879, Val Acc: 0.5050


100%|██████████| 2/2 [00:00<00:00, 57.15it/s]


Epoch 2/70, Loss: 12.4799, Val Loss: 1.3842, Val Acc: 0.5000


100%|██████████| 2/2 [00:00<00:00, 54.05it/s]


...

100%|██████████| 2/2 [00:00<00:00, 29.83it/s]


Epoch 69/70, Loss: 0.0012, Val Loss: 3.8737, Val Acc: 0.7650


100%|██████████| 2/2 [00:00<00:00, 29.74it/s]


Epoch 70/70, Loss: 0.0005, Val Loss: 3.9167, Val Acc: 0.7500


100%|██████████| 2/2 [00:00<00:00, 30.77it/s]


In [ ]:


re_float = re.compile(r"[+-]? *(?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?")
m = re_float.findall("   macro avg     0.8119    0.7800    0.7742       200")
print(m)

['     0.8119', '    0.7800', '    0.7742', '       200']


In [ ]:
accuracy = []
precision = []
recall = []
f1score = []
re_float = re.compile(r"[+-]? *(?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?")
for i in range(10):
    with open(rf'logs\rt2k-bi-gru\fold-{i}.txt', 'rt') as f:
        text = f.readlines()
        for line in text:
            if 'accuracy' in line:
                numbers = re_float.findall(line)
                accuracy.append(float(numbers[0]))
            if 'macro avg' in line:
                numbers = re_float.findall(line)
                precision.append(float(numbers[0]))
                recall.append(float(numbers[1]))
                f1score.append(float(numbers[2]))

print(f'accuracy: {np.mean(accuracy)}')
print(f'precision: {np.mean(precision)}')
print(f'recall: {np.mean(recall)}')
print(f'f1score: {np.mean(f1score)}')

accuracy: 0.7785
precision: 0.7821499999999999
recall: 0.7772
f1score: 0.7766399999999999
